In [1]:
def bottle_filling(fbd,fsbd,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt):
    
    import numpy as np
    import simpy
    
        # Function to generate random breakdown times
    def breakdown_time():
        return np.random.exponential(fbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fbd)  # Breakdown occurence distribution

    def small_breakdown_time():
        return np.random.exponential(fsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fsbd)  # Breakdown occurence distribution
    
    # Function to generate random breakdown times
    def labeller_breakdown_time():
        return np.random.exponential(lbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lbd)  # Breakdown occurence distribution
    
    def labeller_small_breakdown_time():
        return np.random.exponential(lsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lsbd)  # Breakdown occurence distribution

    def maintenance_occurence_time():
        return mo  #Maintenance occurence distribution
    
    # Function to generate random repair times    
    def repair_time():
        return np.random.exponential(frt)  #filler Repair time distribution
    def small_repair_time():
        return np.random.exponential(fsrt)  #filler small Repair time distribution
    
    def labeller_repair_time():
        return np.random.exponential(lrt)  #labeller Repair time distribution
    def labeller_small_repair_time():
        return np.random.exponential(lsrt)   #labeller small Repair time distribution
    def maintenance_time():
        return mt  # maintenance time 
    
    # Bottling process
    def bottling(env, machine, stats):
        while True:
            yield env.timeout(1)  # Bottling takes 1 time unit

            if not machine.working:
                # If machine is not working due to breakdown, skip this cycle
                continue   
            stats['total_bottles'] += 1
            
    # Breakdown process
    def breakdown(env, machine, stats):
        while True:
            yield env.timeout(breakdown_time())
            if machine.working:            
                machine.working = False
                stats['filler_big_breakdowns'] += 1
                machine.down.succeed()  # Trigger the repair process
    def small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(small_breakdown_time())
            if machine.working:            
                machine.working = False
                stats['filler_small_breakdowns'] += 1
                machine.down2.succeed()  # Trigger the repair process
    # Breakdown process
    def labeller_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_breakdown_time())
            if machine.working:            
                machine.working = False
                stats['labeller_big_breakdowns'] += 1
                machine.down3.succeed()  # Trigger the repair process
    def labeller_small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_small_breakdown_time())
            if machine.working:            
                machine.working = False
                stats['labeller_small_breakdowns'] += 1
                machine.down4.succeed()  # Trigger the repair process
    def maintenance(env, machine, stats):
        while True:
            yield env.timeout(maintenance_occurence_time())
            if machine.working:            
                machine.working = False
                stats['Total maintenance occurences'] += 1
                machine.down5.succeed()  # Trigger the repair process
                
        # Repair process
    def repair(env, machine, stats):
        while True:
            yield machine.down        
            repair_duration = repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration 
            stats['big_filler_downtime'] += repair_duration
            machine.working = True
            machine.down = env.event()  # Reset repair event

    def small_repair(env, machine, stats):
        while True:
            yield machine.down2        
            repair_duration = small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration 
            stats['small_filler_downtime'] += repair_duration
            machine.working = True
            machine.down2 = env.event()  # Reset repair event        

    # Repair process
    def labeller_repair(env, machine, stats):
        while True:
            yield machine.down3       
            repair_duration = labeller_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration 
            stats['labeller_big_downtime'] += repair_duration
            machine.working = True
            machine.down3 = env.event()  # Reset repair event

    def labeller_small_repair(env, machine, stats):
        while True:
            yield machine.down4        
            repair_duration = labeller_small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration 
            stats['labeller_small_downtime'] += repair_duration
            machine.working = True
            machine.down4 = env.event()  # Reset repair event

    def maintenance_action(env, machine, stats):
        while True:
            yield machine.down5       
            repair_duration = maintenance_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration 
            stats['Total_maintenance_time'] += repair_duration
            machine.working = True
            machine.down5 = env.event()  # Reset repair event
            
    # Simulation setup
    env = simpy.Environment()
    machine = simpy.Resource(env, capacity=1)  # Bottling machine resource
    machine.working = True  # Initially machine is working
    machine.down = env.event()  # Create event for filler breakdown
    machine.down2 = env.event()  # Create event for small filler breakdown
    machine.down3 = env.event()  # Create event for labeller breakdown
    machine.down4 = env.event()  # Create event for small labeller breakdown
    machine.down5 = env.event()  # Create event for machine maintenance

    stats = {'total_bottles': 0, 'filler_big_breakdowns': 0, 'total_downtime': 0, 'filler_small_breakdowns': 0, 
             'labeller_small_breakdowns': 0, 'labeller_big_breakdowns': 0 ,'small_filler_downtime': 0, 'big_filler_downtime': 0
            , 'Total_maintenance_time': 0,'labeller_small_downtime':0,'labeller_big_downtime':0,'Total maintenance occurences':0}

    # Start processes
    env.process(bottling(env, machine, stats))

    #Filler inputs:
    env.process(repair(env, machine, stats))
    env.process(breakdown(env, machine, stats))
    env.process(small_repair(env, machine, stats))
    env.process(small_breakdown(env, machine, stats))
    #Labeller inputs:
    env.process(labeller_repair(env, machine, stats))
    env.process(labeller_breakdown(env, machine, stats))
    env.process(labeller_small_repair(env, machine, stats))
    env.process(labeller_small_breakdown(env, machine, stats))
    #Maintenance inputs:
    env.process(maintenance_action(env, machine, stats))
    env.process(maintenance(env, machine, stats))

    # Run simulation
    simulation_time = 168*60*6 #Simulation time in minutes
    env.run(until=simulation_time)  # Run the simulation for amount of simulation time

    return stats['total_bottles']

#How to use the function: bottle_filling(fbd,fsmb,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt); 
#fbd:filler breakdown rate ,fsmb: filler small breakdown rate,lbd:labeller breakdown rate,
#lsbd: labeller small breakdown rate ,mo: maintenance rate ,frt: filler big breakdown repair time,
#fsrt: filler small breakdown repair time,lrt: labeller repair time,lsrt: labeller small breakdown repair time,
#mt: maintenance time. All inputs are epxonential with the expected value being the input, except for maintenance time and 
#maintenance frequency, these are a specific time between maintenance and a specific time for maintenance, since they usually
#can be determined by operations. Everything is in minutes, the output is the amount of minutes of filling line operation in 4 weeks

In [6]:
bottle_filling(24*60,60,36*60,45,24*60*7,120,2,55,4,4*60)

47318